In [1]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [2]:
# load data
import constants
df = pd.read_csv("kddcup.data_10_percent.gz", names = constants.names)

In [3]:
# Categorical features to numeric labels

from sklearn import preprocessing
le_dicts = {}

for categorical_name in constants.categorical_names:
    le = preprocessing.LabelEncoder()
    le.fit(df[categorical_name])
    le_dicts[categorical_name] = dict(zip(le.transform(le.classes_), le.classes_))
    df[categorical_name + '_num'] = le.fit_transform(df[categorical_name])

df['cible'] = df.label.apply(lambda label: 1 if label == 'normal.' else 0)
df['label_four'] = df.label.apply(lambda label: constants.label_to_four_attack_class[label])
df['label_four_num'] = df.label_four.apply(lambda label: constants.five_classes_to_num[label])
df.drop(constants.categorical_names + ['label','label_four_num','label_four'], axis=1, inplace=True)
df.shape

(494021, 42)

In [4]:
# head of data (5)
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,protocol_type_num,service_num,flag_num,cible
0,0,181,5450,0,0,0,0,0,1,0,...,0.11,0.0,0.0,0.0,0.0,0.0,1,22,9,1
1,0,239,486,0,0,0,0,0,1,0,...,0.05,0.0,0.0,0.0,0.0,0.0,1,22,9,1
2,0,235,1337,0,0,0,0,0,1,0,...,0.03,0.0,0.0,0.0,0.0,0.0,1,22,9,1
3,0,219,1337,0,0,0,0,0,1,0,...,0.03,0.0,0.0,0.0,0.0,0.0,1,22,9,1
4,0,217,2032,0,0,0,0,0,1,0,...,0.02,0.0,0.0,0.0,0.0,0.0,1,22,9,1


In [5]:
# df to values
df = df.values

In [6]:
# Deep RandomForest
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/kdd.json")
gc = GCForest(config)

In [8]:
# train test
from sklearn.cross_validation import train_test_split
y = df[:,41]
X = df[:,0:41]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
# count class
len(np.unique(y_train))

2

In [10]:
    # X_enc is the concatenated predict_proba result of RandomForest 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    

[ 2018-03-28 08:07:40,511][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 41)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 41)],y_test.shape=(163027,)
[ 2018-03-28 08:07:40,573][cascade_classifier.fit_transform] group_dims=[41]
[ 2018-03-28 08:07:40,574][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-03-28 08:07:40,575][cascade_classifier.fit_transform] group_ends=[41]
[ 2018-03-28 08:07:40,576][cascade_classifier.fit_transform] X_train.shape=(330994, 41),X_test.shape=(163027, 41)
[ 2018-03-28 08:07:40,647][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 41), X_cur_test.shape=(163027, 41)
[ 2018-03-28 08:07:44,104][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-03-28 08:07:47,533][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.97%
[ 2018-03-28 08:07:50,962][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [11]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of RandomForest = {:.6f} %".format(acc * 100))

[ 2018-03-28 08:08:15,544][cascade_classifier.transform] X_groups_test.shape=[(163027, 41)]
[ 2018-03-28 08:08:15,566][cascade_classifier.transform] group_dims=[41]
[ 2018-03-28 08:08:15,567][cascade_classifier.transform] X_test.shape=(163027, 41)
[ 2018-03-28 08:08:15,590][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 41)


Test Accuracy of RandomForest = 99.977918 %


In [12]:
X_train_enc.shape

(330994, 2)

In [13]:
X_test_enc.shape

(163027, 2)

In [14]:
X_train.shape

(330994, 41)

In [15]:
# predict_proba result of RandomForest + data
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-03-28 08:08:17,905][cascade_classifier.transform] X_groups_test.shape=[(163027, 41)]
[ 2018-03-28 08:08:17,928][cascade_classifier.transform] group_dims=[41]
[ 2018-03-28 08:08:17,929][cascade_classifier.transform] X_test.shape=(163027, 41)
[ 2018-03-28 08:08:17,951][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 41)


In [16]:
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [17]:
# cascade RandomForest
config = load_json("./examples/kddgc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-03-28 08:08:20,358][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-03-28 08:08:20,425][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-03-28 08:08:20,426][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-03-28 08:08:20,426][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-03-28 08:08:20,427][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-03-28 08:08:20,501][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-03-28 08:08:23,260][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-03-28 08:08:25,984][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.98%
[ 2018-03-28 08:08:28,855][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [18]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of RandomForest = {:.6f} %".format(acc * 100),i,'fois')

In [19]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/kddgc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()    

[ 2018-03-28 08:10:10,238][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-03-28 08:10:10,307][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-03-28 08:10:10,308][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-03-28 08:10:10,309][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-03-28 08:10:10,310][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-03-28 08:10:10,388][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-03-28 08:10:13,134][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-03-28 08:10:16,009][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.97%
[ 2018-03-28 08:10:19,018][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

[ 2018-03-28 08:12:08,662][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=99.98%
[ 2018-03-28 08:12:11,465][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=99.97%
[ 2018-03-28 08:12:14,260][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=99.98%
[ 2018-03-28 08:12:16,928][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=99.97%
[ 2018-03-28 08:12:19,596][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=99.98%
[ 2018-03-28 08:12:22,403][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.99%
[ 2018-03-28 08:12:25,192][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=99.98%
[ 2018-03-28 08:12:27,974][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.98%


('Test Accuracy of RandomForest = 99.984665 %', 0, 'fois')


[ 2018-03-28 08:14:08,735][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-03-28 08:14:08,801][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-03-28 08:14:08,802][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-03-28 08:14:08,803][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-03-28 08:14:08,804][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-03-28 08:14:08,879][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-03-28 08:14:11,726][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-03-28 08:14:14,622][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.97%
[ 2018-03-28 08:14:17,479][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

[ 2018-03-28 08:16:01,353][cascade_classifier.transform] group_dims=[43]
[ 2018-03-28 08:16:01,354][cascade_classifier.transform] X_test.shape=(163027, 43)
[ 2018-03-28 08:16:01,377][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 43)


...

[ 2018-03-28 11:12:24,994][cascade_classifier.transform] group_dims=[43]
[ 2018-03-28 11:12:24,995][cascade_classifier.transform] X_test.shape=(163027, 43)
[ 2018-03-28 11:12:25,026][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 43)


('Test Accuracy of RandomForest = 99.984052 %', 18, 'fois')


[ 2018-03-28 11:12:33,264][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-03-28 11:12:33,331][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-03-28 11:12:33,332][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-03-28 11:12:33,333][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-03-28 11:12:33,342][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-03-28 11:12:33,416][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-03-28 11:12:45,689][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.98%
[ 2018-03-28 11:12:58,677][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.97%
[ 2018-03-28 11:13:11,935][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

[ 2018-03-28 11:20:56,084][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=99.98%
[ 2018-03-28 11:21:08,033][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=99.97%
[ 2018-03-28 11:21:20,168][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=99.98%
[ 2018-03-28 11:21:32,375][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=99.96%
[ 2018-03-28 11:21:44,446][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=99.97%
[ 2018-03-28 11:21:56,710][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.99%
[ 2018-03-28 11:22:09,176][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=99.97%
[ 2018-03-28 11:22:21,262][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.98%


('Test Accuracy of RandomForest = 99.982212 %', 19, 'fois')


# ('Test Accuracy of RandomForest = 99.985279 %', 1, 'fois')